In [ ]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName('spark')
         .config("spark.executor.memory", "4g")
         .config("spark.executor.cores", "2")
         .config("spark.cores.max", "3")
         .config("spark.driver.memory", "4g")
         .getOrCreate())
sc = spark.sparkContext

In [ ]:
spark

In [ ]:
#rdd_name = sc.textFile("/data/*.csv.gz")

In [ ]:
#xx = rdd_name.map(lambda i: i.split(',')).take(5)
#xx

#
# D = 11 character station identification code
# YEAR/MONTH/DAY = 8 character date in YYYYMMDD format (e.g. 19860529 = May 29, 1986)
# ELEMENT = 4 character indicator of element type 
# DATA VALUE = 5 character data value for ELEMENT 
# M-FLAG = 1 character Measurement Flag 
# Q-FLAG = 1 character Quality Flag 
# S-FLAG = 1 character Source Flag 
# OBS-TIME = 4-character time of observation in hour-minute format (i.e. 0700 =7:00 am)

In [ ]:
from pyspark.sql.types import *

cSchema = StructType([StructField("StationId", StringType()),
                      StructField("Day", LongType()),
                      StructField("ELEMENT", StringType()),
                      StructField("DATA_VALUE", LongType()),
                      StructField("MFlag", StringType()),
                      StructField("QFlag", StringType()),
                      StructField("SFlag", StringType()),
                      StructField("OBSTime", StringType())
                     ])
df = spark.read.csv("/data/*.csv.gz", schema=cSchema)
df.limit(5).toPandas()

In [ ]:
%%time
rc = df.count()
print(rc)

## Benchmark

c5d.2xlarge: 100 Gb data unzipped took... 5min 52s. 2,895,739,748 records


download dataset
```
mkdir data
cd data
kaggle datasets download -d therohk/ireland-historical-news
unzip ireland-historical-news.zip
```

In [10]:
rdd_name = sc.textFile("../data/irishtimes-date-text.csv")
rdd_name.take(5)

['publish_date,headline_category,headline_text',
 '19960102,news,"UUP sees possibility of voting Major out"',
 '19960102,news,"Pubs targeted as curbs on smoking are extended"',
 '19960102,news,"Papers reveal secret links with O\'Neill cabinet"',
 '19960102,news,"Domestic chaos as Italy takes EU presidency"']

In [12]:
from pyspark.sql.types import *

cSchema = StructType([StructField("publish_date", LongType()),
                      StructField("headline_category", StringType()),
                      StructField("headline_text", StringType()),
                     ])
df = spark.read.csv("../data/irishtimes-date-text.csv", schema=cSchema, header=True)
df.limit(5).toPandas()

,publish_date,headline_category,headline_text
0,19960102,news,UUP sees possibility of voting Major out
1,19960102,news,Pubs targeted as curbs on smoking are extended
2,19960102,news,Papers reveal secret links with O'Neill cabinet
3,19960102,news,Domestic chaos as Italy takes EU presidency
4,19960102,news,Learning about the star to which we owe life


In [13]:
%%time
rc = df.count()
print(rc)

1425460
CPU times: user 668 µs, sys: 635 µs, total: 1.3 ms
Wall time: 437 ms


In [19]:
import pyspark.sql.functions as F

df.groupBy('headline_category').count().sort(F.desc('count')).toPandas()

,headline_category,count
0,news,574774
1,sport,153559
2,business,106155
3,opinion.letters,70262
4,opinion,38413
...,...,...
151,business.exchange-rates,3
152,lifestyle.travel.hotels,2
153,lifestyle.abroad.canada,2
154,lifestyle.coffee-vs-gangs,2
